**Run these cells before prompting the model**

In [ ]:
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
# Sometimes after installing the accelerate library, it is still not recognised. If that happens, restart the runtime and rerun the pip command
!pip install --quiet accelerate

To run properly this feature (bitsandbytes) you need to have GPU that supports 8-bit operation modules. Currently, Turing and Ampere GPUs (RTX20s, RTX30s, A40-A100, T4+) are supported, which means on colab we need to use a T4 GPU for this feature. You can check that using this code snippet and make sure you are using a supported GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# Mounts Google drive to access the folders used for I/0
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Inference with the 3 billion parameters BLOOM model**

In [ ]:
# Parameters for the selected model version
name = "bigscience/bloom-3b"
# Specify the max_new_tokens parameter for the model
no_tokens = 50

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Download and load the model. If there is not a suitable GPU in the system, this will not work.
model_8bit = AutoModelForCausalLM.from_pretrained(name, device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(name)

def generate_from_model(model, tokenizer, text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(encoded_input["input_ids"], max_new_tokens = no_tokens, temperature = 2.0, top_k=2000, top_p=0.1)
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

In [ ]:
def process_answer(answer):
    # Step 1: Split the answer into individual sentences. The prompt is discarded.
    sentences = answer.split('\n')
    sentences = sentences[5].split('.') # Change index number from 5 to 1, if no examples included in prompt
    # Remove extra spaces from sentences
    for i in range(0, len(sentences)):
      sentences[i] = sentences[i].strip()
    # Remove elements with no length
    for i in range(0, len(sentences)):
      if (len(sentences[i]) == 0):
        sentences.remove(sentences[i])

    # Step 2: Create a list to store unique sentences
    unique_sentences = []

    # Step 3: Iterate through each sentence and check for duplicates
    for sentence in sentences:
        # Step 4: If the sentence is not in the list, add it to the list; otherwise, skip it
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)

    # Step 5: Join the unique sentences back into a single string
    unique_answer = '. '.join(unique_sentences) + '.\n'

    return unique_answer

**Generate answer from model**

In [ ]:
# File path to example input on google drive
inputFile = "/content/drive/MyDrive/openAI_input/outputAdaptiveSystem.csv"

In [ ]:
# List to store the cleaned triples
triples = []

with open(inputFile) as input:
    for line in input:
      elements = line.strip().split(",")
      # Remove unused number
      elements.pop(-1)
      # Clean the elements
      new_elem = []
      for word in elements:
        new_elem.append(word.replace("food_galmat_1.7.", ""))
      # Convert cleaned list into a tuple
      new_elem = tuple(new_elem)
      triples.append(new_elem)
print(triples)

[('Bakso', 'hasIngredient', 'Noodle', '0.3'), ('Bakso', 'hasIngredient', 'Vermicelli', '0.3'), ('Bakso', 'hasIngredient', 'Celery', '0.3'), ('Bakso', 'isEatenInCountry', 'NationwideInIndonesia', '0.3'), ('Bakso', 'isEatenInCountry', 'ChineseCuisine', '0.3'), ('Bakso', 'isEatenInRegion', 'NeighboringSoutheastAsianCountries', '0.3'), ('Bakso', 'isEatenInRegion', 'NationwideInIndonesia', '0.3'), ('BarnyCakes', 'hasIngredient', 'SpongeCake', '-0.732'), ('BarnyCakes', 'isEatenInCountry', 'France', '-0.732'), ('BarnyCakes', 'isCreatedBy', 'MondelezInternational', '-0.732'), ('BarnyCakes', 'hasDishVariation', 'Chocolate', '-0.732'), ('BarnyCakes', 'hasDishVariation', 'Apple', '-0.732'), ('BaconExplosion', 'hasIngredient', 'Sausage', '-0.268'), ('BaconExplosion', 'hasIngredient', 'Bacon', '-0.268'), ('BaconExplosion', 'isEatenInCountry', 'UnitedStates', '-0.268'), ('BaconExplosion', 'hasMainIngredients', 'Sausage', '-0.268'), ('BaconExplosion', 'hasMainIngredients', 'Bacon', '-0.268'), ('Bacon

In [ ]:
# Array to save each description
texts = []
for triple in triples:
  subject, predicate, obj, learner_ability = triple
  # Providing the model with examples of the task it needs to perform. This is an example of few-shot prompting.
  text = "Describe the triple (Ajoblanco, country, Spain) in a sentence:\n Ajoblanco is from Spain.\nDescribe the triple (Asam_pedas, region, Malay_Peninsula) in a sentence:\n Asam pedas is from the Malay Peninsula region.\n"
  # To use this model in the zero-shot prompting setting, comment out the previous line
  text += f"Describe the triple ({subject}, {predicate}, {obj}) in a sentence:"
  validText = ""
  # A for loop to attempt to get a valid description out of the model
  for i in range(0,5):
    genText = generate_from_model(model_8bit, tokenizer, text)
    validText = process_answer(genText)
    if ((validText.find(subject)!= -1) and (validText.find(obj)!= -1)):
      break
  texts.append(validText)
print("\n".join(texts))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1509: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Bakso is made of noodle

Bakso is made with Vermicelli

Bakso is made with celery

Bakso is eaten in Indonesia nationwide

Bakso is eaten in China

Bakso is eaten in neighboring Southeast Asian countries

Bakso is eaten in Indonesia nationwide

BarnyCakes has sponge cake as its main ingredient

BarnyCakes is eaten in France

BarnyCakes is created by Mondelez International

BarnyCakes has a dish variation of chocolate

BarnyCakes has a dish variation of apple

Bacon explosion has sausage as an ingredient

Bacon explosion has the ingredient Bacon

Bacon explosion is eaten in the United States

Bacon explosion has main ingredients sausage

Bacon explosion has main ingredients of bacon

Bacon explosion is eaten in Kansas City metropolitan area

Bacon explosion is a main course



In [ ]:
# Save the output of the Bloom model in a text file
outputPath = "/content/drive/MyDrive/BLOOM_output/BLOOMoutput_3_fewShot.txt"
with open(outputPath, "w") as file:
        file.write("\n".join(texts))